In [ ]:
import os
import urllib.request
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dataset_path = "../data/download/"
data_raw = "../data/raw/"


def download_dataset(dataset_path):
    urllib.request.urlretrieve(
        "https://go.criteois.com/criteo-research-attribution-dataset.zip",
        dataset_path + "criteo_attribution_dataset.zip",
    )

    zippath = dataset_path + "criteo_attribution_dataset.zip"
    with zipfile.ZipFile(zippath, "r") as zip_ref:
        zip_ref.extractall(data_raw)


if not os.path.exists(data_raw + "criteo_attribution_dataset.tsv.gz"):
    print("Downloading dataset...")
    download_dataset(dataset_path)

In [ ]:
class Net(nn.Module):
    def __init__(
        self, num_touchpoints, embedding_dim=None, hidden_size=None, out_features=None
    ):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        embedding_dim = (
            embedding_dim if embedding_dim else int(np.sqrt(num_touchpoints))
        )
        hidden_size = hidden_size if hidden_size else int(2 * num_touchpoints)
        out_features = out_features if out_features else hidden_size

        self.embeding = nn.Embedding(
            num_embeddings=num_touchpoints, embedding_dim=embedding_dim
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=hidden_size, num_layers=1
        )
        self.fc1 = nn.Linear(in_features=hidden_size, out_features=out_features)
        self.fc2 = nn.Linear(out_features, 1, bias=False)
        self.fc3 = nn.Linear(hidden_size, 1, bias=False)
        self.bias1 = nn.Parameter(torch.zeros(1, 1))
        self.bias2 = nn.Parameter(torch.zeros(1, 1))
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.PReLU()

    def forward(self, x):
        e = self.embeding(x[0])
        h, _ = self.lstm(e.view(1, 1, -1))
        v = self.tanh(self.fc1(h[0]))
        a = self.softmax(self.fc2(v) - self.bias1 * x[1])
        s = torch.dot(a, h)
        p = self.sigmoid(self.relu(self.fc3(s)) + self.bias2)
        return p